In [ ]:
# Demand Forecasting Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Prophet kütüphanesi (kuruluysa)
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    print("⚠️ Prophet kütüphanesi bulunamadı. Sadece ML modelleri kullanılacak.")
    PROPHET_AVAILABLE = False

print("📈 TALEP TAHMİNİ ANALİZİ")
print("="*50)

# Veriyi yükle
df = pd.read_csv('results/processed_data.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 1. ÜRÜN BAZLI TALEP TAHMİNİ
print("\n🛍️ ÜRÜN BAZLI TALEP TAHMİNİ")
print("-" * 40)

# Günlük ürün satışlarını hazırla
daily_product_sales = df.groupby(['StockCode', df['InvoiceDate'].dt.date]).agg({
    'Quantity': 'sum',
    'TotalAmount': 'sum'
}).reset_index()

daily_product_sales.columns = ['StockCode', 'Date', 'Daily_Quantity', 'Daily_Revenue']
daily_product_sales['Date'] = pd.to_datetime(daily_product_sales['Date'])

# En çok satan ürünleri seç (analiz için)
top_products = df.groupby('StockCode')['Quantity'].sum().nlargest(20).index.tolist()
print(f"📊 En çok satan {len(top_products)} ürün için tahmin yapılacak")

# Ürün bazlı özellikler oluştur
def create_product_features(product_data):
    """Ürün için zaman serisi özellikleri"""

    # Tarih özellikleri
    product_data['Year'] = product_data['Date'].dt.year
    product_data['Month'] = product_data['Date'].dt.month
    product_data['DayOfWeek'] = product_data['Date'].dt.dayofweek
    product_data['DayOfYear'] = product_data['Date'].dt.dayofyear
    product_data['IsWeekend'] = product_data['DayOfWeek'].isin([5, 6]).astype(int)

    # Lag özellikleri (geçmiş satış verileri)
    product_data = product_data.sort_values('Date')
    product_data['Lag_1'] = product_data['Daily_Quantity'].shift(1)
    product_data['Lag_7'] = product_data['Daily_Quantity'].shift(7)
    product_data['Lag_30'] = product_data['Daily_Quantity'].shift(30)

    # Hareketli ortalamalar
    product_data['MA_7'] = product_data['Daily_Quantity'].rolling(window=7).mean()
    product_data['MA_30'] = product_data['Daily_Quantity'].rolling(window=30).mean()

    # Trend özellikleri
    product_data['Trend_7'] = product_data['Daily_Quantity'].rolling(window=7).apply(
        lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) == 7 else 0
    )

    return product_data

# 2. Prophet ile zaman serisi tahmini (eğer mevcut)
def prophet_forecast(product_data, stock_code, days_ahead=30):
    """Prophet ile ürün talep tahmini"""

    if not PROPHET_AVAILABLE:
        return None, None

    # Prophet için veri formatı
    prophet_data = product_data[['Date', 'Daily_Quantity']].copy()
    prophet_data.columns = ['ds', 'y']
    prophet_data = prophet_data.dropna()

    if len(prophet_data) < 30:  # Minimum veri gereksinimi
        return None, None

    # Model oluştur
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05
    )

    try:
        model.fit(prophet_data)

        # Gelecek tarihler
        future = model.make_future_dataframe(periods=days_ahead)
        forecast = model.predict(future)

        return model, forecast

    except Exception as e:
        print(f"⚠️ Prophet hatası ({stock_code}): {e}")
        return None, None

# 3. Machine Learning ile talep tahmini
def ml_forecast(product_data, stock_code):
    """Random Forest ile talep tahmini"""

    # Özellikleri oluştur
    featured_data = create_product_features(product_data)

    # Eksik değerleri temizle
    ml_data = featured_data.dropna()

    if len(ml_data) < 50:  # Minimum veri gereksinimi
        return None, None, None

    # Özellikler ve hedef değişken
    feature_cols = ['Year', 'Month', 'DayOfWeek', 'DayOfYear', 'IsWeekend',
                   'Lag_1', 'Lag_7', 'Lag_30', 'MA_7', 'MA_30', 'Trend_7']

    X = ml_data[feature_cols]
    y = ml_data['Daily_Quantity']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False, random_state=42
    )

    # Random Forest modeli
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )

    rf_model.fit(X_train, y_train)

    # Tahminler
    y_pred = rf_model.predict(X_test)

    # Performans metrikleri
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Feature_Importance': dict(zip(feature_cols, rf_model.feature_importances_))
    }

    return rf_model, metrics, ml_data

# Ana tahmin döngüsü
product_forecasts = {}
ml_results = {}

print("🔄 Ürün tahminleri hesaplanıyor...")

for i, stock_code in enumerate(top_products[:5]):  # İlk 5 ürün için örnek
    print(f"  📦 {stock_code} işleniyor... ({i+1}/{min(5, len(top_products))})")

    product_data = daily_product_sales[
        daily_product_sales['StockCode'] == stock_code
    ].copy()

    if len(product_data) < 30:
        continue

    # Prophet tahmini
    if PROPHET_AVAILABLE:
        prophet_model, prophet_forecast_data = prophet_forecast(product_data, stock_code)
        if prophet_forecast_data is not None:
            product_forecasts[stock_code] = {
                'prophet_model': prophet_model,
                'prophet_forecast': prophet_forecast_data
            }

    # ML tahmini
    ml_model, ml_metrics, ml_data = ml_forecast(product_data, stock_code)
    if ml_model is not None:
        ml_results[stock_code] = {
            'model': ml_model,
            'metrics': ml_metrics,
            'data': ml_data
        }

print(f"✅ {len(ml_results)} ürün için ML modeli oluşturuldu")
if PROPHET_AVAILABLE:
    print(f"✅ {len(product_forecasts)} ürün için Prophet modeli oluşturuldu")

# 4. MÜŞTERİ BAZLI TALEP TAHMİNİ
print("\n👥 MÜŞTERİ BAZLI TALEP TAHMİNİ")
print("-" * 40)

# Müşteri-zaman bazlı satış verileri
customer_monthly = df.groupby(['Customer ID', df['InvoiceDate'].dt.to_period('M')]).agg({
    'TotalAmount': 'sum',
    'Quantity': 'sum',
    'Invoice': 'nunique'
}).reset_index()

customer_monthly.columns = ['Customer ID', 'Month', 'Monthly_Revenue', 'Monthly_Quantity', 'Order_Count']
customer_monthly['Month'] = customer_monthly['Month'].dt.to_timestamp()

# En aktif müşteriler
top_customers = df.groupby('Customer ID')['TotalAmount'].sum().nlargest(50).index.tolist()

def create_customer_features(customer_data):
    """Müşteri için özellik mühendisliği"""

    customer_data = customer_data.sort_values('Month')

    # Zaman özellikleri
    customer_data['Year'] = customer_data['Month'].dt.year
    customer_data['MonthNum'] = customer_data['Month'].dt.month
    customer_data['Quarter'] = customer_data['Month'].dt.quarter

    # Lag özellikleri
    customer_data['Revenue_Lag1'] = customer_data['Monthly_Revenue'].shift(1)
    customer_data['Revenue_Lag3'] = customer_data['Monthly_Revenue'].shift(3)
    customer_data['Quantity_Lag1'] = customer_data['Monthly_Quantity'].shift(1)

    # Trend
    customer_data['Revenue_MA3'] = customer_data['Monthly_Revenue'].rolling(window=3).mean()
    customer_data['Revenue_Growth'] = customer_data['Monthly_Revenue'].pct_change()

    return customer_data

# Müşteri bazlı modeller
customer_ml_results = {}

print("🔄 Müşteri tahminleri hesaplanıyor...")

for i, customer_id in enumerate(top_customers[:10]):  # İlk 10 müşteri
    print(f"  👤 Müşteri {customer_id} işleniyor... ({i+1}/10)")

    customer_data = customer_monthly[
        customer_monthly['Customer ID'] == customer_id
    ].copy()

    if len(customer_data) < 6:  # Minimum 6 aylık veri
        continue

    # Özellikleri oluştur
    featured_data = create_customer_features(customer_data)
    ml_data = featured_data.dropna()

    if len(ml_data) < 4:
        continue

    # Model özellikleri
    feature_cols = ['Year', 'MonthNum', 'Quarter', 'Revenue_Lag1',
                   'Revenue_Lag3', 'Quantity_Lag1', 'Revenue_MA3']

    # Sadece mevcut özellikleri kullan
    available_features = [col for col in feature_cols if col in ml_data.columns]

    if len(available_features) < 3:
        continue

    X = ml_data[available_features]
    y = ml_data['Monthly_Revenue']

    if len(X) < 4:
        continue

    # Basit train-test (son 2 ay test)
    if len(X) > 4:
        X_train = X[:-2]
        X_test = X[-2:]
        y_train = y[:-2]
        y_test = y[-2:]
    else:
        X_train = X[:-1]
        X_test = X[-1:]
        y_train = y[:-1]
        y_test = y[-1:]

    # Model
    rf_customer = RandomForestRegressor(n_estimators=50, random_state=42)
    rf_customer.fit(X_train, y_train)

    y_pred = rf_customer.predict(X_test)

    customer_ml_results[customer_id] = {
        'model': rf_customer,
        'features': available_features,
        'mae': mean_absolute_error(y_test, y_pred),
        'actual_last': y_test.iloc[-1] if len(y_test) > 0 else 0,
        'predicted_last': y_pred[-1]
    }

print(f"✅ {len(customer_ml_results)} müşteri için model oluşturuldu")

# 5. SONUÇ ÖZETİ
print(f"\n📊 GENEL SONUÇLAR")
print("="*50)

if ml_results:
    print("🛍️ ÜRÜN MODELLERİ:")
    for stock_code, results in list(ml_results.items())[:3]:
        metrics = results['metrics']
        print(f"  📦 {stock_code}:")
        print(f"    • MAE: {metrics['MAE']:.2f}")
        print(f"    • RMSE: {metrics['RMSE']:.2f}")
        print(f"    • R²: {metrics['R2']:.3f}")

        # En önemli özellikler
        top_features = sorted(
            metrics['Feature_Importance'].items(),
            key=lambda x: x[1], reverse=True
        )[:3]
        print(f"    • Top özellikler: {', '.join([f[0] for f in top_features])}")

if customer_ml_results:
    print(f"\n👥 MÜŞTERİ MODELLERİ:")
    avg_mae = np.mean([r['mae'] for r in customer_ml_results.values()])
    print(f"  • Ortalama MAE: {avg_mae:.2f}")
    print(f"  • Toplam modellenen müşteri: {len(customer_ml_results)}")

# Sonuçları kaydet
results_summary = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'product_models': len(ml_results),
    'customer_models': len(customer_ml_results),
    'prophet_available': PROPHET_AVAILABLE
}

if ml_results:
    # Ürün model performansları
    product_performance = []
    for stock_code, results in ml_results.items():
        metrics = results['metrics']
        product_performance.append({
            'StockCode': stock_code,
            'MAE': metrics['MAE'],
            'RMSE': metrics['RMSE'],
            'R2': metrics['R2']
        })

    pd.DataFrame(product_performance).to_csv('results/product_forecast_performance.csv', index=False)

if customer_ml_results:
    # Müşteri model performansları
    customer_performance = []
    for customer_id, results in customer_ml_results.items():
        customer_performance.append({
            'Customer_ID': customer_id,
            'MAE': results['mae'],
            'Features_Used': len(results['features']),
            'Last_Actual': results['actual_last'],
            'Last_Predicted': results['predicted_last']
        })

    pd.DataFrame(customer_performance).to_csv('results/customer_forecast_performance.csv', index=False)

print(f"\n✅ Talep tahmini analizi tamamlandı!")
print(f"📁 Performans raporları 'results/' klasöründe")
print(f"🎯 Ürün modelleri: {len(ml_results)}")
print(f"🎯 Müşteri modelleri: {len(customer_ml_results)}")

# Örnek tahmin kodu
prediction_example = '''
# GELECEK TAHMİNLERİ İÇİN ÖRNEK KOD:

# Ürün için gelecek 7 günlük tahmin
def predict_product_demand(stock_code, model_data, days=7):
    if stock_code not in ml_results:
        return "Model bulunamadı"

    model = ml_results[stock_code]['model']
    last_data = ml_results[stock_code]['data'].iloc[-1]

    predictions = []
    for day in range(days):
        # Özellikleri hazırla (son verilerden)
        features = [last_data[col] for col in feature_cols]
        pred = model.predict([features])[0]
        predictions.append(max(0, pred))  # Negatif olmayacak şekilde

    return predictions

# Müşteri için gelecek ay tahmini
def predict_customer_revenue(customer_id):
    if customer_id not in customer_ml_results:
        return "Model bulunamadı"

    model = customer_ml_results[customer_id]['model']
    features = customer_ml_results[customer_id]['features']

    # Son verilerden özellik hazırla
    # ... feature engineering ...

    next_month_revenue = model.predict([feature_vector])[0]
    return max(0, next_month_revenue)
'''

print(f"\n💡 Tahmin kullanım örneği kodu hazır!")